# Genome Assembly: 5. Assembly Quality Assessment

## Overview
Evaluating assembly quality is crucial. This notebook covers:

1. **Contiguity metrics**: N50, L50, NG50, etc.
2. **Accuracy metrics**: Comparing to reference (when available)
3. **Completeness**: What fraction of the genome is assembled?
4. **Coverage analysis**: Is coverage uniform?
5. **Repeat resolution**: How well were repeats handled?
6. **Comparing assemblies**: Which is better?

**Philosophy**: No single metric tells the whole story—use multiple metrics together.

## 1. Contiguity Metrics

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from collections import defaultdict
from typing import List, Tuple, Dict

def calculate_contiguity_metrics(contig_lengths: List[int]) -> dict:
    """
    Calculate standard contiguity metrics.
    
    Key metrics:
    - N50: Length of the contig for which all contigs >= this length comprise 50% of the genome
    - N75, N90: Same but for 75% and 90%
    - L50: Number of contigs needed to achieve N50
    - NGA50: Like N50 but measured against assembly length (genome-agnostic)
    """
    if not contig_lengths:
        return {}
    
    sorted_lengths = sorted(contig_lengths, reverse=True)
    total_length = sum(sorted_lengths)
    
    def get_nx_lx(x: float) -> Tuple[int, int]:
        """Get N{x} and L{x}"""
        cumsum = 0
        for i, length in enumerate(sorted_lengths):
            cumsum += length
            if cumsum >= (total_length * x / 100):
                return length, i + 1
        return 0, len(sorted_lengths)
    
    n50, l50 = get_nx_lx(50)
    n75, l75 = get_nx_lx(75)
    n90, l90 = get_nx_lx(90)
    
    return {
        'num_contigs': len(sorted_lengths),
        'total_length': total_length,
        'mean_length': np.mean(sorted_lengths),
        'median_length': np.median(sorted_lengths),
        'min_length': min(sorted_lengths),
        'max_length': max(sorted_lengths),
        'std_dev': np.std(sorted_lengths),
        'n50': n50,
        'l50': l50,
        'n75': n75,
        'l75': l75,
        'n90': n90,
        'l90': l90,
        'sorted_lengths': sorted_lengths
    }

def print_contiguity_report(metrics: dict):
    """
    Print a formatted contiguity report.
    """
    print(f"""
╔════════════════════════════════════════════════════════════╗
║              ASSEMBLY CONTIGUITY METRICS                   ║
╚════════════════════════════════════════════════════════════╝

Basic Statistics:
  Number of contigs:        {metrics['num_contigs']:>8}
  Total length:             {metrics['total_length']:>8,} bp
  Mean length:              {metrics['mean_length']:>8.1f} bp
  Median length:            {metrics['median_length']:>8.0f} bp
  Std. deviation:           {metrics['std_dev']:>8.1f} bp
  Min/Max length:           {metrics['min_length']:>8}/{metrics['max_length']:<8} bp

N-Statistics (Nx = contig length for which contigs >= this length comprise x% of assembly):
  N50 (50%):                {metrics['n50']:>8} bp  (L50={metrics['l50']})
  N75 (75%):                {metrics['n75']:>8} bp  (L75={metrics['l75']})
  N90 (90%):                {metrics['n90']:>8} bp  (L90={metrics['l90']})

Interpretation:
  • N50 is the most commonly used metric for assembly quality
  • Higher N50 is generally better
  • N50 > 50,000 bp is good for bacterial genomes
  • N50 > 1,000,000 bp is good for mammalian genomes
  • L50 tells you how many large contigs you have
""")

# Test with example assemblies
test_assembly1 = [100000, 95000, 80000, 75000, 70000, 60000, 50000, 40000, 30000, 20000]
test_assembly2 = [150000, 140000, 130000, 5000, 4000, 3000, 2000, 1000, 500, 100]
test_assembly3 = [20000] * 100  # Fragmented assembly

metrics1 = calculate_contiguity_metrics(test_assembly1)
metrics2 = calculate_contiguity_metrics(test_assembly2)
metrics3 = calculate_contiguity_metrics(test_assembly3)

print("ASSEMBLY 1: Balanced contigs")
print_contiguity_report(metrics1)

print("\n" + "="*60)
print("ASSEMBLY 2: Few large + many small (typical)")
print_contiguity_report(metrics2)

print("\n" + "="*60)
print("ASSEMBLY 3: Highly fragmented")
print_contiguity_report(metrics3)

print("\n" + "="*60)
print("COMPARISON")
print(f"Assembly 1 N50: {metrics1['n50']} (good balance)")
print(f"Assembly 2 N50: {metrics2['n50']} (very good)")
print(f"Assembly 3 N50: {metrics3['n50']} (fragmented)")

## 2. Visualization of Contig Size Distribution

In [ ]:
def plot_contig_distribution(metrics: dict, title: str = "Contig Size Distribution"):
    """
    Plot contig size distribution and cumulative plot.
    """
    lengths = metrics['sorted_lengths']
    
    fig, axes = plt.subplots(1, 3, figsize=(15, 4))
    
    # 1. Histogram of contig sizes
    ax = axes[0]
    ax.hist(lengths, bins=30, color='steelblue', edgecolor='black', log=True)
    ax.axvline(metrics['n50'], color='red', linestyle='--', linewidth=2, label=f"N50={metrics['n50']}")
    ax.axvline(metrics['median_length'], color='green', linestyle='--', linewidth=2, label=f"Median={metrics['median_length']:.0f}")
    ax.set_xlabel('Contig Length (bp)')
    ax.set_ylabel('Number of Contigs (log scale)')
    ax.set_title('Contig Length Distribution')
    ax.legend()
    ax.grid(alpha=0.3)
    
    # 2. Cumulative length plot
    ax = axes[1]
    cumsum = np.cumsum(lengths)
    x = np.arange(1, len(lengths) + 1)
    ax.plot(x, cumsum, color='steelblue', linewidth=2, marker='o', markersize=3)
    ax.axhline(sum(lengths) * 0.5, color='red', linestyle='--', alpha=0.5, label='50%')
    ax.axhline(sum(lengths) * 0.75, color='orange', linestyle='--', alpha=0.5, label='75%')
    ax.axhline(sum(lengths) * 0.9, color='purple', linestyle='--', alpha=0.5, label='90%')
    ax.set_xlabel('Number of Contigs')
    ax.set_ylabel('Cumulative Length (bp)')
    ax.set_title('Cumulative Contig Length')
    ax.legend()
    ax.grid(alpha=0.3)
    
    # 3. Log-scale cumulative plot
    ax = axes[2]
    ax.loglog(x, cumsum, color='steelblue', linewidth=2, marker='o', markersize=3)
    ax.axhline(sum(lengths) * 0.5, color='red', linestyle='--', alpha=0.5, label='50%')
    ax.axvline(metrics['l50'], color='red', linestyle='--', alpha=0.5)
    ax.set_xlabel('Number of Contigs (log scale)')
    ax.set_ylabel('Cumulative Length (log scale)')
    ax.set_title(f"L50 = {metrics['l50']}")
    ax.legend()
    ax.grid(alpha=0.3, which='both')
    
    fig.suptitle(title, fontsize=14, fontweight='bold')
    plt.tight_layout()
    plt.show()

plot_contig_distribution(metrics1, "Assembly 1: Balanced")
plot_contig_distribution(metrics2, "Assembly 2: Few Large + Many Small")
plot_contig_distribution(metrics3, "Assembly 3: Fragmented")

## 3. Accuracy Assessment (Reference-Based)

In [ ]:
def simple_alignment_identity(query: str, subject: str, window: int = 100) -> float:
    """
    Simple sequence identity calculation (not a full alignment).
    Uses sliding window approach.
    """
    if len(query) == 0 or len(subject) == 0:
        return 0.0
    
    matches = 0
    total = 0
    
    for i in range(min(len(query), len(subject))):
        if query[i] == subject[i]:
            matches += 1
        total += 1
    
    return (matches / total * 100) if total > 0 else 0.0

def assess_accuracy(contigs: List[str], reference: str, 
                   coverage_map: Dict = None) -> dict:
    """
    Assess assembly accuracy against a reference genome.
    
    Metrics:
    - Sequence identity: % of bases matching reference
    - Assembly size vs reference size
    - Structural mismatches: inversions, duplications, etc.
    """
    
    # Calculate basic statistics
    assembled_length = sum(len(c) for c in contigs)
    reference_length = len(reference)
    
    # Calculate coverage
    reference_positions_covered = set()
    for contig in contigs:
        for i in range(len(reference) - len(contig) + 1):
            if reference[i:i+len(contig)] == contig:
                reference_positions_covered.update(range(i, i+len(contig)))
    
    completeness = len(reference_positions_covered) / reference_length * 100
    
    # Calculate identity
    assembled_seq = ''.join(contigs)
    identity = simple_alignment_identity(assembled_seq, reference)
    
    return {
        'assembled_length': assembled_length,
        'reference_length': reference_length,
        'length_ratio': assembled_length / reference_length,
        'completeness_percent': completeness,
        'sequence_identity': identity,
        'assembly_contigs': len(contigs),
    }

# Test accuracy assessment
print("\nAccuracy Assessment (Reference-Based)")
print("="*70)

ref_seq = "ATGCGATCGATCGATCGATCGATCGATCGATCG" * 50  # 1700 bp reference

# Perfect assembly
perfect_assembly = [ref_seq]

# Assembly with errors
error_assembly = [ref_seq[:500], ref_seq[600:1000], ref_seq[1100:]]  # Missing regions

# Assembly with sequence errors
error_seq = list(ref_seq)
for i in [100, 200, 500, 800]:  # Introduce errors
    if i < len(error_seq):
        bases = ['A', 'T', 'G', 'C']
        error_seq[i] = [b for b in bases if b != error_seq[i]][0]
identity_assembly = [''.join(error_seq)]

accuracy_perfect = assess_accuracy(perfect_assembly, ref_seq)
accuracy_gaps = assess_accuracy(error_assembly, ref_seq)
accuracy_errors = assess_accuracy(identity_assembly, ref_seq)

for name, accuracy in [("Perfect Assembly", accuracy_perfect),
                      ("Assembly with Gaps", accuracy_gaps),
                      ("Assembly with Errors", accuracy_errors)]:
    print(f"\n{name}:")
    print(f"  Assembled vs Reference: {accuracy['assembled_length']:,}/{accuracy['reference_length']:,} bp ({accuracy['length_ratio']*100:.1f}%)")
    print(f"  Completeness: {accuracy['completeness_percent']:.1f}% of reference covered")
    print(f"  Sequence Identity: {accuracy['sequence_identity']:.2f}%")

## 4. Coverage Analysis

In [ ]:
def map_reads_to_contigs(reads: List[str], contigs: List[str], 
                        min_match: int = 20) -> Dict[int, List[int]]:
    """
    Simple read mapping to contigs.
    Returns dict mapping contig_id -> list of coverage at each position.
    """
    coverage_maps = {}
    
    for c_idx, contig in enumerate(contigs):
        coverage = [0] * len(contig)
        
        for read in reads:
            # Try to find where read maps
            for i in range(len(contig) - min_match + 1):
                if contig[i:].startswith(read[:min_match]):
                    # Found a match
                    for j in range(min(len(read), len(contig) - i)):
                        coverage[i + j] += 1
                    break
        
        coverage_maps[c_idx] = coverage
    
    return coverage_maps

def analyze_coverage(coverage_maps: Dict[int, List[int]]) -> dict:
    """
    Analyze coverage distribution.
    """
    all_coverage = []
    for contig_coverage in coverage_maps.values():
        all_coverage.extend([c for c in contig_coverage if c > 0])
    
    if not all_coverage:
        return {'mean': 0, 'median': 0, 'std': 0, 'min': 0, 'max': 0}
    
    return {
        'mean': np.mean(all_coverage),
        'median': np.median(all_coverage),
        'std': np.std(all_coverage),
        'min': np.min(all_coverage),
        'max': np.max(all_coverage),
        'cv': np.std(all_coverage) / np.mean(all_coverage) if np.mean(all_coverage) > 0 else 0
    }

def plot_coverage(coverage_maps: Dict[int, List[int]], title: str = "Read Coverage Profile"):
    """
    Plot coverage for all contigs.
    """
    n_contigs = len(coverage_maps)
    
    if n_contigs == 1:
        fig, ax = plt.subplots(1, 1, figsize=(12, 4))
        axes = [ax]
    elif n_contigs <= 4:
        fig, axes = plt.subplots(n_contigs, 1, figsize=(12, 3*n_contigs))
        if n_contigs == 1:
            axes = [axes]
    else:
        fig, axes = plt.subplots(4, 1, figsize=(12, 12))
    
    for idx, (c_id, coverage) in enumerate(sorted(coverage_maps.items())[:4]):
        ax = axes[idx]
        x = np.arange(len(coverage))
        ax.fill_between(x, 0, coverage, color='steelblue', alpha=0.7)
        ax.plot(x, coverage, color='darkblue', linewidth=1)
        
        if coverage:
            mean_cov = np.mean([c for c in coverage if c > 0])
            ax.axhline(mean_cov, color='red', linestyle='--', alpha=0.5, label=f'Mean={mean_cov:.1f}x')
        
        ax.set_ylabel('Coverage (x)')
        ax.set_title(f'Contig {c_id} Coverage')
        ax.legend()
        ax.grid(alpha=0.3)
    
    fig.suptitle(title, fontsize=14, fontweight='bold')
    plt.tight_layout()
    plt.show()

# Test coverage analysis
print("\nCoverage Analysis")
print("="*70)

test_contigs = [
    "ATGCGATCGATCGATCGATCGATCGATCGATCGATCGATCGATCGATCGATCGATCGATCG",
    "TCGACGTAGCTAGCTAGCTAGCTAGCTAGCTAGCTAGCTAGCTAGCTAGCTAGCTAGCTA"
]

# Generate reads with variable coverage
import random
random.seed(42)
test_reads = []
for _ in range(50):
    contig = random.choice(test_contigs)
    start = random.randint(0, max(0, len(contig) - 30))
    test_reads.append(contig[start:start+30])

cov_maps = map_reads_to_contigs(test_reads, test_contigs, min_match=15)
cov_analysis = analyze_coverage(cov_maps)

print(f"Coverage Statistics:")
print(f"  Mean: {cov_analysis['mean']:.2f}x")
print(f"  Median: {cov_analysis['median']:.2f}x")
print(f"  Std Dev: {cov_analysis['std']:.2f}x")
print(f"  Min/Max: {cov_analysis['min']:.0f}x / {cov_analysis['max']:.0f}x")
print(f"  Coefficient of Variation: {cov_analysis['cv']:.3f}")

if cov_analysis['cv'] < 0.2:
    print("  ✓ Coverage is uniform (good)")
elif cov_analysis['cv'] < 0.5:
    print("  ~ Coverage is moderately variable")
else:
    print("  ✗ Coverage is highly variable (potential problems)")

plot_coverage(cov_maps, "Test Assembly Coverage")

## 5. Repeat Resolution Assessment

In [ ]:
def detect_repeats_in_assembly(contigs: List[str], min_length: int = 50) -> dict:
    """
    Detect potential repeat sequences and duplications in the assembly.
    """
    from collections import Counter
    
    # Find all k-mers across contigs
    k = 31  # Use relatively large k to be specific
    kmer_counts = Counter()
    kmer_locations = defaultdict(list)  # kmer -> list of (contig_id, position)
    
    for c_idx, contig in enumerate(contigs):
        for i in range(len(contig) - k + 1):
            kmer = contig[i:i+k]
            kmer_counts[kmer] += 1
            kmer_locations[kmer].append((c_idx, i))
    
    # Find k-mers that appear multiple times
    repeated_kmers = {kmer: count for kmer, count in kmer_counts.items() if count > 1}
    
    # Find contigs with high repeat content
    repeat_content = {}
    for c_idx, contig in enumerate(contigs):
        repeat_bases = 0
        for i in range(len(contig) - k + 1):
            kmer = contig[i:i+k]
            if kmer_counts[kmer] > 1:
                repeat_bases += 1
        repeat_content[c_idx] = (repeat_bases * k) / len(contig) * 100 if contig else 0
    
    return {
        'repeated_kmers': len(repeated_kmers),
        'max_repeat_count': max(repeated_kmers.values()) if repeated_kmers else 0,
        'repeat_content': repeat_content,
        'high_repeat_contigs': [c for c, pct in repeat_content.items() if pct > 30]
    }

print("\nRepeat Resolution Assessment")
print("="*70)

# Create test assembly with repeats
repeat_unit = "ATGCGATCGATCGATCGATCG"
unique1 = "TTTTAAAA"
unique2 = "GGGGCCCC"

contig_with_repeat = unique1 + repeat_unit * 3 + unique2  # Contains repeat
contig_unique = "AAATTTGGGCCCAAATTTGGG" * 5

test_contigs_repeat = [contig_with_repeat, contig_unique]

repeat_analysis = detect_repeats_in_assembly(test_contigs_repeat)

print(f"Repeat Analysis:")
print(f"  Repeated 31-mers found: {repeat_analysis['repeated_kmers']}")
print(f"  Max repeat count: {repeat_analysis['max_repeat_count']}x")
print(f"\nRepeat content by contig:")
for c_idx, pct in repeat_analysis['repeat_content'].items():
    print(f"  Contig {c_idx}: {pct:.1f}% repeat-derived bases")
    if pct > 30:
        print(f"             ⚠ High repeat content (assembly may be fragmented)")

print(f"\nContigs with high repeat content: {repeat_analysis['high_repeat_contigs']}")

## 6. Comparing Multiple Assemblies

In [ ]:
def compare_assemblies(assemblies: dict) -> dict:
    """
    Compare multiple assemblies comprehensively.
    assemblies: dict mapping assembly_name -> list of contigs
    """
    results = {}
    
    for name, contigs in assemblies.items():
        lengths = [len(c) for c in contigs]
        metrics = calculate_contiguity_metrics(lengths)
        
        results[name] = {
            'num_contigs': metrics['num_contigs'],
            'total_length': metrics['total_length'],
            'n50': metrics['n50'],
            'l50': metrics['l50'],
            'mean_length': metrics['mean_length'],
            'max_length': metrics['max_length']
        }
    
    return results

def print_comparison_table(comparisons: dict):
    """
    Print comparison of assemblies.
    """
    print(f"""
╔═════════════════════════════════════════════════════════════════════════════════════╗
║                        ASSEMBLY COMPARISON                                           ║
╚═════════════════════════════════════════════════════════════════════════════════════╝

{'Assembly':<20} {'Contigs':<10} {'Total BP':<15} {'N50':<12} {'L50':<8} {'Max':<12} {'Mean':<10}
{'─'*95}
""")
    
    for name in sorted(comparisons.keys()):
        c = comparisons[name]
        print(f"{name:<20} {c['num_contigs']:<10} {c['total_length']:<15,} {c['n50']:<12,} {c['l50']:<8} {c['max_length']:<12,} {c['mean_length']:<10.0f}")

print("\nAssembly Comparison Example")
print("="*70)

# Create three different assemblies
assembly_a = test_assembly1  # Balanced
assembly_b = test_assembly2  # Few large + many small
assembly_c = test_assembly3  # Fragmented

assemblies = {
    "Assembly A (Balanced)": [[str(i)] * l for l in assembly_a],
    "Assembly B (Few Large)": [[str(i)] * l for l in assembly_b],
    "Assembly C (Fragmented)": [[str(i)] * l for l in assembly_c]
}

comps = compare_assemblies(assemblies)
print_comparison_table(comps)

print("\nRanking by Key Metrics:")
print(f"\nBest N50:")
best_n50 = max(comps.items(), key=lambda x: x[1]['n50'])
print(f"  {best_n50[0]}: {best_n50[1]['n50']} bp")

print(f"\nBest contiguity (lowest L50):")
best_l50 = min(comps.items(), key=lambda x: x[1]['l50'])
print(f"  {best_l50[0]}: {best_l50[1]['l50']} contigs")

print(f"\nLongest single contig:")
best_max = max(comps.items(), key=lambda x: x[1]['max_length'])
print(f"  {best_max[0]}: {best_max[1]['max_length']} bp")

print(f"\n🏆 Overall best: {best_n50[0]} (highest N50 is most important)")

## 7. Choosing an Assembler

In [ ]:
assembler_guide = """
CHOOSING THE RIGHT ASSEMBLER
═════════════════════════════════════════════════════════════════════════════════════

┌─ SHORT READS (Illumina, up to ~250bp) ─────────────────────────────────────────────┐
│                                                                                     │
│  SPADES (SPAdes - Genome Assembler for SPecial PurposEs) - MOST POPULAR            │
│  • De Bruijn graph based                                                           │
│  • Uses multiple k-mer sizes (e.g., 21, 33, 55, 77)                               │
│  • Excellent for bacteria, fungi, small eukaryotes                                │
│  • Good error correction                                                           │
│  • CPU: ~10-50GB RAM, reasonable time                                              │
│  • Install: conda install -c bioconda spades                                       │
│  • Use: spades.py -1 reads_R1.fastq -2 reads_R2.fastq -o output_dir               │
│                                                                                     │
│  VELVET                                                                             │
│  • Original De Bruijn graph assembler                                              │
│  • Simple, fast, but needs tuning                                                  │
│  • Install: conda install -c bioconda velvet                                       │
│                                                                                     │
│  ABYSS                                                                              │
│  • De Bruijn graph, parallelizable                                                 │
│  • Good for large genomes                                                          │
│  • Install: conda install -c bioconda abyss                                        │
│                                                                                     │
└─────────────────────────────────────────────────────────────────────────────────────┘

┌─ LONG READS (PacBio, Oxford Nanopore, >10kb) ──────────────────────────────────────┐
│                                                                                     │
│  FLYE                                                                               │
│  • De Bruijn-like for long reads                                                   │
│  • Fast, accurate                                                                  │
│  • Install: conda install -c bioconda flye                                         │
│  • Use: flye --pacbio-raw reads.fastq -o output_dir -t 32                         │
│                                                                                     │
│  CANU                                                                               │
│  • Overlap-Layout-Consensus                                                        │
│  • Accurate even with 5-10% error rate                                             │
│  • Install: conda install -c bioconda canu                                         │
│                                                                                     │
└─────────────────────────────────────────────────────────────────────────────────────┘

┌─ HYBRID (Mix of short + long reads) ────────────────────────────────────────────────┐
│                                                                                     │
│  SPADES (with long reads)                                                           │
│  • Use: spades.py -1 short_R1.fastq -2 short_R2.fastq --pacbio long_reads.fastq    │
│                                                                                     │
│  UNICYCLER                                                                          │
│  • Hybrid assembler specifically designed for hybrid assemblies                    │
│  • Install: conda install -c bioconda unicycler                                    │
│                                                                                     │
└─────────────────────────────────────────────────────────────────────────────────────┘

SELECTING BY ORGANISM
───────────────────────────────────────────────────────────────────────────────────────

Bacteria / Archaea (typically 2-10 Mb, haploid):
  → SPAdes (default choice)
  → For high quality: FLYE if long reads available
  → For very large (>50 Mb): Consider MEGAHIT

Fungi (typically 10-100 Mb, often haploid):
  → SPAdes
  → FLYE for long reads

Plants (typically 100-10,000 Mb, diploid/polyploid):
  → FLYE (long reads preferred)
  → SPAdes for short reads only (may struggle with large genomes)
  → Consider: PLATANUS, ALLPATHS-LG

Animals (typically 100-10,000 Mb):
  → FLYE or Canu (long reads)
  → SPAdes (short reads, for smaller animals)
  → Hybrid: UNICYCLER, PLATANUS

Viruses (typically <500 kb):
  → Any assembler works
  → SPAdes is common choice

KEY QUALITY THRESHOLDS
───────────────────────────────────────────────────────────────────────────────────────

Bacterial assembly quality expectations:
  N50 > 50 kb    : Good
  N50 > 200 kb   : Very Good
  N50 > 1 Mb     : Excellent (near complete)

Eukaryotic assembly quality (human-sized):
  N50 > 100 kb   : Reasonable (can improve scaffolding)
  N50 > 1 Mb     : Good
  N50 > 10 Mb    : Very Good
  N50 > 50 Mb    : Excellent

READ COVERAGE RECOMMENDATIONS
───────────────────────────────────────────────────────────────────────────────────────

Sequencing depth (coverage) needed:
  Bacteria:      10-20x sufficient
  Fungi:         20-50x recommended
  Plants:        30-50x (diploid), 50-100x (polyploid)
  Vertebrates:   30-50x typical
  Complex:       50-100x for confident assembly

Higher coverage helps with:
  • Error correction
  • Resolving repeats
  • Low-coverage regions
  • Polyploidy

Diminishing returns above 100x for most simple genomes.
"""

print(assembler_guide)

## 8. Quality Assessment Checklist

In [ ]:
def generate_qa_report(assembly: List[str], reference: str = None, reads: List[str] = None) -> str:
    """
    Generate a comprehensive quality assessment report.
    """
    lengths = [len(c) for c in assembly]
    metrics = calculate_contiguity_metrics(lengths)
    
    report = f"""
╔═════════════════════════════════════════════════════════════╗
║            ASSEMBLY QUALITY ASSESSMENT REPORT                ║
╚═════════════════════════════════════════════════════════════╝

[✓] CONTIGUITY
  • Number of contigs: {metrics['num_contigs']}
  • Total length: {metrics['total_length']:,} bp
  • N50: {metrics['n50']} bp (L50={metrics['l50']})
  • Longest contig: {metrics['max_length']} bp
  • Mean length: {metrics['mean_length']:.0f} bp
  • Status: """
    
    if metrics['n50'] > 100000:
        report += "✓ EXCELLENT\n"
    elif metrics['n50'] > 50000:
        report += "✓ GOOD\n"
    elif metrics['n50'] > 10000:
        report += "~ FAIR\n"
    else:
        report += "✗ FRAGMENTED\n"
    
    # Repeat analysis
    repeat_analysis = detect_repeats_in_assembly(assembly)
    report += f"""
[?] REPEAT CONTENT
  • Repeated k-mers: {repeat_analysis['repeated_kmers']}
  • Max repeat frequency: {repeat_analysis['max_repeat_count']}x
  • High-repeat contigs: {len(repeat_analysis['high_repeat_contigs'])}
  • Status: """
    
    if len(repeat_analysis['high_repeat_contigs']) == 0:
        report += "✓ Low repeat content\n"
    else:
        report += f"~ Some contigs have high repeat content\n"
    
    if reference:
        accuracy = assess_accuracy(assembly, reference)
        report += f"""
[R] ACCURACY (vs reference)
  • Assembled length: {accuracy['assembled_length']:,} bp
  • Reference length: {accuracy['reference_length']:,} bp
  • Coverage: {accuracy['length_ratio']*100:.1f}% of reference
  • Completeness: {accuracy['completeness_percent']:.1f}%
  • Sequence identity: {accuracy['sequence_identity']:.2f}%
  • Status: """
        
        if accuracy['completeness_percent'] > 95:
            report += "✓ Complete\n"
        elif accuracy['completeness_percent'] > 80:
            report += "✓ Good coverage\n"
        else:
            report += "~ Incomplete\n"
    
    report += f"""
["] RECOMMENDATIONS
  • For publication/production use:
    - Validate contigs with independent data (PCR, qPCR, other sequencing)
    - Fill gaps where possible
    - Polish with long reads if available
    - Check for chimerism (correct assembly errors)
    
  • To improve assembly:
"""
    
    if metrics['n50'] < 50000:
        report += "    - Try longer reads or higher coverage\n"
    if len(repeat_analysis['high_repeat_contigs']) > 0:
        report += "    - Use longer reads to resolve repeat regions\n"
    if reference and accuracy['completeness_percent'] < 95:
        report += "    - Increase sequencing coverage\n"
    
    report += f"""
═══════════════════════════════════════════════════════════════
"""
    
    return report

# Generate sample report
test_assembly = [test_assembly1[0], test_assembly1[1], test_assembly1[2]]  # Take a few contigs
report = generate_qa_report(test_assembly[:3])
print("\n" + report)

## Summary

**Assembly Quality Metrics:**
- **Contiguity**: N50/L50 most important; higher is better
- **Completeness**: What % of the genome is covered?
- **Accuracy**: Sequence identity to reference (if available)
- **Coverage**: Is coverage uniform? Indicates assembly problems if not
- **Repeats**: High repeat content → fragmented assembly

**Key Takeaways:**
1. No single metric is perfect—use all together
2. N50 is most commonly reported but context matters
3. Compare multiple assemblies if possible
4. Consider your organism and goals when evaluating
5. Validation against independent data is essential for production genomes

**Tools for Quality Assessment:**
- `QUAST`: Automated QA if reference is available
- `Busco`: Completeness check against conserved genes
- `AssemblyStats`: Simple statistics
- Custom scripts (like this notebook!)

---

This concludes the genome assembly series! You now understand:
1. ✓ Fundamentals (k-mers, De Bruijn graphs)
2. ✓ Read preprocessing (QC, trimming, filtering)
3. ✓ Assembly algorithms (greedy vs De Bruijn)
4. ✓ Scaffolding and contig joining
5. ✓ Quality assessment

Next steps: Try it yourself with real data from SRA!